In [1]:
%matplotlib
import numpy as np
import csv
import json

Using matplotlib backend: <object object at 0x0000027640CB7580>


In [2]:
def inScattering(sctData):
    
    
    
    for i in range(len(sctData)):
        for j in range(len(sctData[0])):
            if i == j:
                sctData[i][j] = 0
    
    return sctData

In [3]:
def outScattering(sctData):
    
    outScat = np.zeros_like(sctData)
    
    for j in range(len(sctData[0])):
        sum = 0
        for i in range(len(sctData)):
            if i != j:
                sum += sctData[i][j]
                
        
        outScat[j][j] = sum 
        
    return outScat
                

In [4]:
def readCSV(filename):
    
    absorptions = []
    fissions = []
    chis = []
    SctData = []
    
    with open(filename, 'r') as file:
        
        csvreader = csv.reader(file)
        next(csvreader)#Title for Data
        next(csvreader)#Titles of Cross-Sections
        i=0
        
        
        for row in csvreader:
            #print(row)
            try:
                absorptions.append(float(row[1]))
                fissions.append(float(row[2]))
                chis.append(float(row[3]))
                i+=1
            except:
                
                break
        
        next(csvreader) #skips straight to next data
        
            
        length = len(absorptions)+1
        #print(length)
        
        for row in csvreader:
            #print(row)
            
            SctData.append([float(number) for number in row[1:length]])
            
    #print(absorptions)
    #print(fissions)
    #print(chis)
    #print(SctData)
    
    absMatr = np.diag(absorptions)
    
   
    
    fissMatr = np.zeros((len(fissions),len(fissions)))
    for i in range(0, len(fissMatr)):
        for j in range(0,len(fissMatr[0])):
            fissMatr[i][j] = fissions[j]*chis[i]
    
    
    
    #print(fissMatr)
    
    
    
    
    data = {}
    data["absorption"] = absMatr.tolist()
    data["fission"] = fissMatr.tolist()
    
    data["inScat"] = inScattering(SctData)
    data["outScat"] = outScattering(SctData).tolist()
    
    
    
    """
    data["inScat"] = np.array()
    data["outScat"] = np.array(SctData).tolist()
    """
    
    return data

In [5]:
def getMigration(absorps, outScat, inScat):
    
    migration = absorps + outScat - inScat
    
    return migration

In [6]:
def powerIterate(B_init, value):
    
    
    k_prev = 1
    s = (len(B_init), 1)
    
    phi_prev = np.ones(s)
    
    
    
    for i in range(0, value):
        
        phi_prev_num = np.matmul(B_init, phi_prev) #numerator of eq calculating new phi
        
        phi_new = phi_prev_num / np.linalg.norm(phi_prev_num) #values for next iteration
         
        phi_new_num = np.matmul(B_init, phi_new)# B times new flux
        
        k_new_num = np.matmul(np.transpose(phi_new_num), phi_new)#numerator for new  k value
        k_new_den = np.matmul(np.transpose(phi_new), phi_new)#denominator for new k value
        
        k_new = k_new_num / k_new_den
        
        k_prev = k_new
        
        phi_prev = phi_new
        
    return k_prev, phi_prev    

In [10]:
def main(filename):
    
    data = readCSV(filename)
    
    outScat, inScat = data["outScat"], data["inScat"]
    
    absorps = data["absorption"]
    fissions = data["fission"]
    
    migration = getMigration(np.array(absorps), np.array(outScat), np.array(inScat))
    
    B_init = np.matmul(np.linalg.inv(migration), np.array(fissions))
    #Calculating all eigenvalues using computer package
    eigenvals, eigenvectors = np.linalg.eig(B_init)
    print(eigenvals,eigenvectors)
    #get rid of complex in both
    eigenvals = eigenvals.tolist()
    eigenvectors = eigenvectors.tolist()
    
    eigenvals = [i.real for i in eigenvals]
    for i in range(len(eigenvectors)): 
        for j in range(len(eigenvectors[0])):
        
            eigenvectors[i][j] = eigenvectors[i][j].real 
    
    
    #calculate largest eigenval/eigenvector 
    k_final, phi_final = powerIterate(B_init, 2)#2 iterations
    
    eigs = {}
    eigs["valsComp"] = eigenvals #from the linalg package
    eigs["vecsComp"] = eigenvectors
    eigs["valPower"] = k_final.tolist()
    eigs["vecPower"] = phi_final.tolist()
    
    return eigs

In [11]:
def writeToOutput(inputFile):
    eigs = main(inputFile)
    inputData = readCSV(inputFile)
    
    
    
    outputFile = inputFile[5:-4] + ".txt"
    
    with open(outputFile, 'w+') as f:
        json.dump([inputData, eigs], f)

In [12]:
main("InputDataCSVGroup8.csv")
#writeToOutput("InputDataCSVGroup2.csv")

[ 0.00000000e+00+0.00000000e+00j  1.09003100e+00+0.00000000e+00j
 -6.24639424e-17+0.00000000e+00j  1.09446495e-17+7.44624414e-18j
  1.09446495e-17-7.44624414e-18j -2.81127678e-18+7.77575628e-18j
 -2.81127678e-18-7.77575628e-18j  5.06864619e-18+0.00000000e+00j] [[-0.98273382+0.j         -0.26158725+0.j          0.28819332+0.j
   0.01667315+0.18326782j  0.01667315-0.18326782j -0.04438974+0.50508758j
  -0.04438974-0.50508758j -0.25825287+0.j        ]
 [ 0.06596855+0.j         -0.3441239 +0.j          0.91351983+0.j
  -0.16487252-0.1085734j  -0.16487252+0.1085734j   0.00714675-0.12165263j
   0.00714675+0.12165263j -0.06132389+0.j        ]
 [ 0.14990534+0.j         -0.78197886+0.j          0.21908363+0.j
   0.9310343 +0.j          0.9310343 -0.j          0.83835139+0.j
   0.83835139-0.j          0.95490768+0.j        ]
 [ 0.05374405+0.j         -0.280355  +0.j          0.04495739+0.j
   0.22755484+0.0411089j   0.22755484-0.0411089j   0.10009914+0.08251757j
   0.10009914-0.08251757j  0.12529

{'valsComp': [0.0,
  1.090030997354204,
  -6.246394243332582e-17,
  1.0944649490533659e-17,
  1.0944649490533659e-17,
  -2.811276782895975e-18,
  -2.811276782895975e-18,
  5.068646192585621e-18],
 'vecsComp': [[-0.9827338208670579,
   -0.26158724776311115,
   0.2881933244405346,
   0.01667314796719826,
   0.01667314796719826,
   -0.04438973622022237,
   -0.04438973622022237,
   -0.25825287242944],
  [0.06596854844762808,
   -0.3441239021957946,
   0.913519830102881,
   -0.16487251558560811,
   -0.16487251558560811,
   0.007146753894456307,
   0.007146753894456307,
   -0.06132388754645341],
  [0.14990533908725992,
   -0.7819788590262127,
   0.2190836318880507,
   0.9310343042690911,
   0.9310343042690911,
   0.8383513897945695,
   0.8383513897945695,
   0.9549076753598097],
  [0.05374404961570546,
   -0.2803549950510686,
   0.044957389128464,
   0.22755483653115113,
   0.22755483653115113,
   0.10009913538856463,
   0.10009913538856463,
   0.12529805906284075],
  [0.0427996373029088,
  